<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/main/order_preserving_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Order-preserving RAG**

* Das lässt manche an der Notwendigkeit von Retrieval-Augmented Generation (RAG) zweifeln. Bei RAG werden Informationen üblicherweise zunächst in einer Vektordatenbank gespeichert und dynamisch bei jeder Anfrage abgerufen.

* Eine neue Studie von Nvidia-Forschenden stellt diese Annahme, dass man RAG mit wachsenden Kontextfenstern nicht brauchen würde, in Frage. Sie zeigt, dass RAG in Kombination mit einem bestimmten Mechanismus LLMs mit großen Kontextfenstern übertreffen kann.

* Das Paper "In Defense of RAG in the Era of Long-Context Language Models" schlägt konkret einen reihenfolgeerhaltenden RAG-Ansatz (OP-RAG, OP steht für order-preserving) vor, der die ursprüngliche Reihenfolge der abgerufenen Teile (Chunks) im LLM-Kontext beibehält. Dies steht im Gegensatz zu herkömmlichen RAG-Methoden, die die Chunks meist nach absteigender Relevanz anordnen.

* **Die Studie untersuchte auch den Einfluss der Kontextlänge auf die Leistung von OP-RAG. Die Ergebnisse zeigten, dass die Antwortqualität mit zunehmender Kontextlänge zunächst verbessert, dann aber abnahm.**

* Paper: [In Defense of RAG in the Era of Long-Context Language Models](https://arxiv.org/abs/2409.01666)

* https://the-decoder.de/studie-laesst-zweifel-an-nutzen-grosser-kontextfenster-aufkommen/

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import faiss

# Load your LLM
model_name = "your-llm-model" # Replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Load your knowledge base (replace with your own data)
dataset = load_dataset("your-dataset")

# Create a FAISS index for efficient retrieval
index = faiss.IndexFlatL2(768)  # Assuming 768-dimensional embeddings
passages = [example['text'] for example in dataset['train']]
index.add(passages)

# Order-preserving RAG function
def order_preserving_rag(query, k=3):
    # Retrieve relevant passages
    _, passage_indices = index.search(query, k)

    # Keep the original order of the retrieved passages
    retrieved_passages = [passages[i] for i in passage_indices[0]]

    # Combine the query and retrieved passages for the LLM
    context = f"Query: {query}\nContext: {' '.join(retrieved_passages)}"

    # Generate a response
    response = generator(context, max_length=100)

    return response

# Example usage
query = "What is the capital of France?"
response = order_preserving_rag(query)
print(response)

1. **Load LLM and Knowledge Base:**
   - Replace `'your-llm-model'` and `'your-dataset'` with your actual LLM and dataset.
   - We're assuming a dataset with a 'text' field containing the knowledge base passages.

2. **FAISS Index:**
   - FAISS is used for efficient similarity search to retrieve relevant passages.
   - You'll likely need to preprocess your data and generate embeddings before creating the index.

3. **`order_preserving_rag` Function:**
   - Takes a `query` and optionally the number of passages to retrieve (`k`).
   - Retrieves relevant passages using FAISS, maintaining their original order.
   - Combines the query and retrieved passages into a context for the LLM.
   - Generates a response using the LLM.

4. **Example Usage:**
   - Demonstrates how to use the function to answer a question.

**Notes**

- **Order Preservation:** The core idea is to keep the retrieved passages in the same order they appear in the knowledge base. This can be crucial for tasks where the order of information matters (e.g., summarizing a historical event).
- **FAISS:** FAISS is used for efficient retrieval, but you might need to adapt it to your specific embedding method.
- **LLM Prompting:** The way you combine the query and retrieved passages in the context can significantly impact the LLM's performance. Experiment with different prompt formats.

**Considerations**

- **Preprocessing and Embeddings:** You'll need to preprocess your knowledge base and generate embeddings suitable for FAISS.
- **Error Handling:** Add error handling for cases where retrieval fails or the LLM generates unexpected responses.
- **Evaluation:** Carefully evaluate the performance of your Order-preserving RAG system on your specific task.